In [1]:
from utilities import *

df = load_file("./Data/SPY_FULL.csv", date_format="%d/%m/%Y")
df = add_rsi(df)
print(df.dtypes)
print(df.head())
print(df.tail())

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
rsi                 float64
dtype: object
   timestamp      open      high      low     close   volume  rsi
0 1999-11-19  142.4062  142.9687  142.000  142.5000  4832100  NaN
1 1999-11-22  142.4375  143.0000  141.500  142.4687  4155400  NaN
2 1999-11-23  142.8437  142.8437  140.375  141.2187  5918000  NaN
3 1999-11-24  140.7500  142.4375  140.000  141.9687  4459700  NaN
4 1999-11-26  142.4687  142.8750  141.250  141.4375  1693900  NaN
      timestamp    open    high       low   close    volume        rsi
6352 2025-02-24  602.02  603.03  596.4900  597.21  50737213  44.002908
6353 2025-02-25  597.15  597.89  589.5600  594.24  58266472  41.409659
6354 2025-02-26  595.93  599.58  591.8556  594.54  43321578  41.782877
6355 2025-02-27  596.85  598.02  584.6500  585.05  74196664  34.332585
6356 2025-02-28  585.56  594.72  582.44

In [ ]:
INITIAL_CASH = 10**6
POSITION_SIZE = 0.01

stats = {
    "PROFIT_LOSS_LOG": [],
    "TOTAL_TRADES": 0,
    "WINNING_TRADES": 0,
    "LONG_ARRAY_IDX": [],
    "SHORT_ARRAY_IDX": []
}

open_positions = []
for i, row in df.iterrows():
    rsi = row["rsi"]

    if rsi < 35:
        open_positions.append(
            {"type": "LONG", "entry": row["close"], "entry_date": row["timestamp"], "entry_rsi": rsi}
        )
        stats["LONG_ARRAY_IDX"].append(i)
        

    for idx in reversed(range(len(open_positions))):
        position = open_positions[idx]

        if position["type"] == "LONG":
            if rsi >= 80:
                stats, log = close_trade(position, row, stats)
                verbose_log += log
                open_positions.pop(idx)

In [ ]:
win_rate = round((stats["WINNING_TRADES"] / stats["TOTAL_TRADES"]) * 100, 2)
loss_rate = 100 - win_rate
buy_and_hold = percent_change(df['close'].iloc[0], df['close'].iloc[-1])

print(f"Total Trades: {stats["TOTAL_TRADES"]}")
print(f"Win Rate: {win_rate}%")
print(f"Loss Rate: {loss_rate}%")
print(f"P/L per Trade: {average(stats["PROFIT_LOSS_LOG"]):.2f}%")
print(f"Buy & Hold Return: {buy_and_hold}%")

write_to_file("trade_log.txt", verbose_log)

Total Trades: 733
Win Rate: 78.31%
Total Profit/Loss: 19602.74%
Buy & Hold Return: 316.97%


rough idea: use past longer period standard dev to identify if market is sideways, then use RSI to entry exit

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, 
                    subplot_titles=("Stock Price", "RSI"))

fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['close'], mode='lines', name='Stock Price', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['rsi'], mode='lines', name='RSI', line=dict(color='red')),
    row=2, col=1
)

fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=35, y1=35,
              line=dict(color="grey", width=2, dash="dot"), row=2, col=1)

fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=70, y1=70,
              line=dict(color="grey", width=2, dash="dot"), row=2, col=1)

for idx in stats["long_array_idx"]:
    fig.add_annotation(dict(
        x=df['timestamp'][idx],
        y=df['close'][idx],
        xref="x1", yref="y1",
        text="⬆",
        showarrow=False,
        font=dict(color="#03fc17", size=10)
    ))

for idx in stats["short_array_idx"]:
    fig.add_annotation(dict(
        x=df['timestamp'][idx],
        y=df['close'][idx],
        xref="x1", yref="y1",
        text="⬇",
        showarrow=False,
        font=dict(color="#fc0303", size=10)
    ))

fig.update_layout(height=600, title="Stock Price and RSI", showlegend=False)
fig.show()